# Classification with Deep Learning (CNNs)

In this lesson, we learn how to solve a classification problem through a **Deep Learning** approach based on *Convolutional Neural Networks* (CNNs).

**It is absolutely recommended to read the documentation relating to the functions and methods used!**
Usually, it is sufficient typing on Google the name of the function (and eventually the name of the library used).

Let's import some **libraries**. We will use *TensorFlow* and *Keras* as Deep Learning framework.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

### Data


1.   Upload the `.zip` file containing the *Euclid* dataset (Deep Learning version!) → this is for **training** and **validation**
2.   Upload the `.zip` file containing the *Euclid* dataset ("standard" version) → this is for **testing**
3.   Unzip all files using the following commands. Dataset folders will appear in `/content`

In [ ]:
!unzip -q Euclid_dataset_DL.zip -d /content

In [ ]:
!unzip -q Euclid_dataset.zip -d /content

### Data Loaders

We can use already developed methods to load data for our DL solution.

Also in this case, we split the dataset with **80% of data in the training** and the remaining **20% in the validation set**.

**Tools**:
   * `image_dataset_from_directory()`: generates a dataset from image files in a directory, that yields batches of images from the subdirectories present in our directory.

**Training** dataset

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='Euclid_dataset_DL',
    labels='inferred',
    label_mode='categorical',
    class_names=['triangle', 'rectangle', 'square', 'rhombus'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

**Validation** dataset

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='Euclid_dataset_DL',
    labels='inferred',
    label_mode='categorical',
    class_names=['triangle', 'rectangle', 'square', 'rhombus'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

**Testing** set

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='Euclid_dataset',
    labels='inferred',
    label_mode='categorical',
    class_names=['triangle', 'rectangle', 'square', 'rhombus'],
    color_mode='rgb',
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=1821,
    validation_split=0,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
)

### Model architecture
It's time to define the architecture of our *Convolutional Neural Network* (CNN), and we have two options:

1.   Defining our **own architecture**.
2.   Using one of the architecture proposed in the literature.  

In our exercitation, we use the `MobileNet` CNN. We will use this architecture **pre-trained** on the `Imagenet` dataset (we are going to download the weights of the network from the official storage).





In [ ]:
# Our model
model = tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
    )

**Problem**: as you can see, the pre-trained model has 1000 classes, i.e. the final dense layer has 1000 neurons, while in our classification task we have only 4 classes.

We need to adapt the architecture. We can remove the last layer and create a new dense layer with only 4 neurons.

**Tools**:

*   [**Sequential model**](https://keras.io/guides/sequential_model/) in TensorFlow
  *   We can define a model as a sequence of layers (`output = Layer()(input)`)
  *   Its layers are accessible via the `layers` attribute
  *   Every layer has an `input` and `output` attribute. These attributes can be used to do neat things, like quickly creating a model that extracts the outputs of all intermediate layers.



In [ ]:
# Create a layer where input is the output of the second last layer
output = Dense(4, activation='softmax', name='predictions')(model.layers[-2].output)

# Then create the corresponding model
model = Model(model.input, output)

In [ ]:
model.summary()

Here we define:

*   number of **epochs**
*   model saving
*   **optimizer**
*   **loss** function

We also define the **callbacks**: a callback is an object (method) that can perform actions at various stages of training (*e.g.* at the start or end of an epoch, before or after a single batch, etc).

We need to **compile** the model before the training phase.



In [ ]:
epochs = 5

callbacks = [
    # to save the model after every epoch
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
    # logging
    tf.keras.callbacks.TensorBoard(log_dir="logs", write_graph=True, write_images=False, update_freq="epoch",)
]

model.compile(
    optimizer=keras.optimizers.SGD(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

### Training
If the training is too slow, remember to run the code on a machine equipped with a GPU.

On Colab, click on **Runtime → Change Runtime type → GPU** (from the drop-down menu)

In [ ]:
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

### Model Evaluation

Tensorflow method to easily evaluate the trained network on a test dataset.

In [ ]:
model_trained = keras.models.load_model('/content/save_at_1.h5')

model_trained.evaluate(
    x=test_ds,
    y=None,
    batch_size=32,
    verbose=True,
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
)

We can load the **TensorBoard** module (provided with Tensorflow) to see our line plots!

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs

### Homework
What if:

*   you do not load a pre-trained model?
*   you change the optimizer?
*   you change the amount of available training data?
*   you change the architecture?

